In [1]:
import json
import os
import pickle
import re
import string

In [6]:
emails = {}
with open(os.getcwd() + '\\Data\\emailsText.pickle', 'rb') as pckl:
    emails = pickle.load(pckl)
pckl.close()
len(emails)

75419

In [8]:
stoplist = []
with open(os.getcwd() + '\\Data\\stoplist.txt') as sl:
    sw = sl.readline()
    
    while sw:
        stoplist.append(sw.strip())
        sw = sl.readline()
sl.close()
# stoplist

In [9]:
emails['inmail.1']

{'label': 1,
 'subject': 'Generic Cialis, branded quality@ ',
 'text': 'Do you feel the pressure to perform and not rising to the occasion??\nTry Viagra.....\nyour anxiety will be a thing of the past and you will\nbe back to your old self.\n'}

In [10]:
from nltk.stem import PorterStemmer

In [13]:
def clean():
    stemmer = PorterStemmer()
    corpus = {}
    count = 1
    for email in emails:
        text = re.sub(r'[^\w\s]', '', emails[email]['text'])
        words = re.split('[^a-zA-Z]', text)
        filtered = []
        
        for word in words:
            word = stemmer.stem(word)
            if word not in stoplist:
                filtered.append(word)
        
        cleaned = ' '.join(filtered)
        corpus[email] = cleaned
        
        if count % 1000 == 0:
            print(count)
            
        count += 1
        
    return corpus

In [14]:
corpus = clean()
len(corpus)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000


75419

In [16]:
with open(os.getcwd() + '\\corpus.pickle', 'wb') as pckl:
    pickle.dump(corpus, pckl)
print('pckld')

pckld


In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import dump_svmlight_file, load_svmlight_file
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [30]:
def selectFeatures():
    contents = []
    labels = []
    
    for email in corpus:
        contents.append(corpus[email])
        labels.append(emails[email]['label'])
        
    vectoriser = CountVectorizer(min_df = 0.02, max_df = 0.95)
    X = vectoriser.fit_transform(contents)
    features = vectoriser.get_feature_names()
    
    f = open(os.getcwd() + '\\unigrams', 'wb')
    dump_svmlight_file(X, labels, f)
    f.close()
    
    with open(os.getcwd()+'\\featureNames.txt', 'w') as f:
        for i in range(0, len(features)):
            f.write(json.dumps({i: features[i]}))
    f.close()
    print('done')

In [31]:
selectFeatures()

done


In [32]:
from sklearn.metrics import mean_squared_error, roc_auc_score

In [43]:
data = load_svmlight_file(os.getcwd() + '\\unigrams')
X = data[0]
y = data[1]
trainX, testX, trainY, testY = train_test_split(X, y, test_size = 0.2)

In [44]:
def train(opt = 1):
    if opt == 1:
        clf = LinearRegression()
    elif opt == 2:
        clf = DecisionTreeClassifier()
    elif opt == 3:
        clf = GaussianNB()
    
    clf.fit(trainX, trainY)
    predY = clf.predict(testX)

    print(roc_auc_score(testY, predY))

In [45]:
train(1)  # linear regression

0.9881627927498703


In [46]:
train(2)  # DT

0.9739861388990896


In [48]:
# train(3)  # naive bayes